In [2]:
# t_loss_bkwd2 vs t_loss_bkwd3 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd2, t_loss_bkwd3
BS, H, N, D = 2, 2, 512, 64 # 2, 2, 5, 4 # 1, 1, 3, 4 #2, 2, 5, 4
vocab_size = 128 #4
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda").to(torch.int32)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
# print(mask)

res2 = t_loss_bkwd2(layers_params, y, mask, None, True, p_gen_aux)
#print(res2[1])
print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_loss_bkwd3(layers_params, y, mask, None, True, p_gen_aux)
#print(res3)
print_res_shapes(res3[0]) 


5
- 2
-- torch.Size([128, 64]) tensor([-0.000855,  0.003516, -0.002200], device='cuda:0')
-- torch.Size([128]) tensor([-0.002048, -0.001721, -0.001041], device='cuda:0')
- 1
-- torch.Size([512, 64]) tensor([ 0.000343, -0.000105, -0.000268], device='cuda:0')
- 10
-- torch.Size([64]) tensor([-2.349084e-07,  4.672267e-06, -3.649368e-07], device='cuda:0')
-- torch.Size([64]) tensor([-6.970960e-05,  1.557064e-05, -3.060383e-06], device='cuda:0')
-- torch.Size([2, 3, 32, 64]) tensor([-6.779784e-06,  3.787625e-05, -4.095639e-05], device='cuda:0')
-- torch.Size([64, 64]) tensor([ 7.367545e-05,  6.718180e-05, -2.441602e-06], device='cuda:0')
-- torch.Size([64]) tensor([-4.768415e-05,  5.973072e-06, -1.383817e-05], device='cuda:0')
-- torch.Size([64]) tensor([-3.136267e-05, -1.356788e-04,  2.052386e-05], device='cuda:0')
-- torch.Size([256, 64]) tensor([-2.353426e-04, -7.052643e-05,  1.298243e-05], device='cuda:0')
-- torch.Size([256]) tensor([-1.375788e-04, -8.888469e-05, -4.294830e-05], device